In [1]:
# rvest 라이브러리를 불러옴
if (!require(rvest)) install.packages('rvest')
library(rvest)

if (!require(tidyverse)) install.packages('tidyverse')
library(tidyverse)

# 날짜와 페이지를 입력하면 조건에 따른 영화 코드를 반환하는 함수를 정의함
get_movie_code <- function(date, page){ # ex) date = 20220502, page = 1
    
    base_url <- 'https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&date='
    target_url <- paste0(base_url, date, '&page=', page)
    
    tables <- target_url %>%
        read_html(encoding = 'UTF-8') %>%
        html_nodes('table')
    
    hrefs <- tables[[1]] %>%
        html_nodes('a') %>%
        html_attr('href')
    
    hrefs_odd <- hrefs[c(TRUE, FALSE)] # 같은 코드이나 서로 다른링크가 짝으로 존재함을 확인했다. 홀수번째 원소만 인덱싱한다.
    codes <- substr(hrefs_odd, unlist(gregexpr('=', hrefs_odd)) + 1, nchar(hrefs_odd)) # 다섯자리 코드와 여섯자리 코드가 혼재하므로 등호를 기준으로 인덱싱한다.
    
    return(codes)
}

# 영화의 코드를 입력하면 영화의 정보를 반환하는 함수를 정의함
get_movie_info <- function(code){
    base_url <- 'https://movie.naver.com/movie/bi/mi/point.nhn?code='
    target_url <- paste0(base_url, code)
    html <- read_html(target_url)
    
    title_unclean <- html %>%
    html_nodes("title") %>%
    html_text()

    title <- substr(title_unclean, 1, unlist(gregexpr(' : 네이버 영화', title_unclean))-1)
    exist <- html %>% html_nodes('dl[class=info_spec]') %>% html_nodes('dt') %>% html_text()

    steps <- html %>%
        html_nodes('dl[class=info_spec]') %>%
        html_nodes('dd')

    # 개요, 감독, 출연, 등급 중 결측값이 존재하는 경우를 대비함
    step1 = NA
    step2 = NA
    step3 = NA
    step4 = NA
    for (i in 1:length(exist)){
        if (exist[i] == '개요()'){
            step1_unclean <- steps[i] %>%
                html_nodes('p') %>%
                    html_nodes('span') %>%
                html_text()
            step1 <- gsub('\\t|\\n|\\r', '', step1_unclean)
        }else if (exist[i] == '감독'){
            step2 <- steps[i] %>%
                html_nodes('p') %>%
                html_text()
        }else if (exist[i] == '출연'){
            step3 <- steps[i] %>%
                html_text()
        }else if (exist[i] == '등급'){
            step4_unclean <- steps[i] %>%
                html_nodes('p') %>%
                html_text()
            step4 <- gsub('\\t|\\n|\\r', '', step4_unclean)
        }
    }
    
    tdt <- html %>%
        html_nodes('div[class=viewing_graph]')
    
    # 성별, 나이별 관람추이가 존재하지 않는 경우 관람객 통계가 존재하지 않으므로 결측값으로 처리함
    if (length(tdt) == 0){
        audience_age_10 <- NA
        audience_age_20 <- NA
        audience_age_30 <- NA
        audience_age_40 <- NA
        audience_age_50 <- NA
        audience_score <- NA
        audience_count <- NA
        audience_male <- NA
        audience_female <- NA
        audience_10 <- NA
        audience_20 <- NA
        audience_30 <- NA
        audience_40 <- NA
        audience_50 <- NA
    } else {
        audi_age <- html %>%
            html_nodes('strong[class=graph_percent]') %>%
            html_text()
        audience_age_10 <- audi_age[1]
        audience_age_20 <- audi_age[2]
        audience_age_30 <- audi_age[3]
        audience_age_40 <- audi_age[4]
        audience_age_50 <- audi_age[5]

        audience_score <- html %>% 
            html_nodes('div[class=grade_audience]') %>%
            html_nodes('div[class=star_score]') %>%
            html_nodes('em') %>%
            html_text() %>% paste(collapse='')

        audience_count <- html %>% 
            html_nodes('div[class=grade_audience]') %>%
            html_nodes('span[class=user_count]') %>%
            html_nodes('em') %>%
            html_text() %>% paste(collapse='')
        
        audience_male <- (html %>%
            html_nodes('div[class=graph_area]') %>%
            html_nodes('div[class=grp_male]') %>%
            html_nodes('strong[class=graph_point]') %>%
            html_text())[2]

        audience_female <- (html %>%
            html_nodes('div[class=graph_area]') %>%
            html_nodes('div[class=grp_female]') %>%
            html_nodes('strong[class=graph_point]') %>%
            html_text())[2]

        audience_age <- html %>%
            html_nodes('div[class=grp_age]') %>%
            html_nodes('strong[class=graph_point]') %>%
            html_text()
        
        audience_10 <- audience_age[6]
        audience_20 <- audience_age[7]
        audience_30 <- audience_age[8]
        audience_40 <- audience_age[9]
        audience_50 <- audience_age[10]
        
    }
    
    netizen_score <- html %>% 
        html_nodes('div[class=grade_netizen]') %>%
        html_nodes('div[class=star_score]') %>%
        html_nodes('em') %>%
        html_text() %>% paste(collapse='')
    
    netizen_count <- html %>% 
        html_nodes('div[class=grade_netizen]') %>%
        html_nodes('span[class=user_count]') %>%
        html_nodes('em') %>%
        html_text() %>% paste(collapse='')
    
    ntz_male <- (html %>%
        html_nodes('div[class=graph_area]') %>%
        html_nodes('div[class=grp_male]') %>%
        html_nodes('strong[class=graph_point]') %>%
        html_text())[1]
    
    ntz_female <- (html %>%
        html_nodes('div[class=graph_area]') %>%
        html_nodes('div[class=grp_female]') %>%
        html_nodes('strong[class=graph_point]') %>%
        html_text())[1]
    
    ntz_age <- html %>%
        html_nodes('div[class=grp_age]') %>%
        html_nodes('strong[class=graph_point]') %>%
        html_text()
    ntz_10 <- ntz_age[1]
    ntz_20 <- ntz_age[2]
    ntz_30 <- ntz_age[3]
    ntz_40 <- ntz_age[4]
    ntz_50 <- ntz_age[5]
    
    return(c(title, code, step1, step2, step3, step4, audience_age_10, audience_age_20, audience_age_30,
     audience_age_40, audience_age_50, netizen_score, netizen_count, ntz_male, ntz_female,
      ntz_10, ntz_20, ntz_30, ntz_40, ntz_50, audience_score, audience_count, audience_male,
     audience_female, audience_10, audience_20, audience_30, audience_40, audience_50))
}

info <- vector('list', 100)
top_100_codes <- c(get_movie_code(20220502, 1), get_movie_code(20220502, 2))
for (i in 1:length(top_100_codes)){ # 한 줄씩 차곡차곡 쌓는다.
    info[[i]] <- get_movie_info(top_100_codes[i])
}

final_info <- do.call('rbind', info)
# 컬럼명을 지정함
colnames(final_info) <- c("title","code","genre","country","runtime","release",
        "director","actor","view_class","audience_age_10","audience_age_20",
        "audience_age_30","audience_age_40","audience_age_50",
        "netizen_score","netizen_count","ntz_male","ntz_female","ntz_10",
        "ntz_20","ntz_30","ntz_40","ntz_50","audience_score",
        "audience_count","audience_male","audience_female","audience_10",
        "audience_20","audience_30","audience_40","audience_50")

write.csv(final_info, 'movie.csv', row.names=T) # 최종 csv파일 생성함

필요한 패키지를 로딩중입니다: rvest

Warning message:
"패키지 'rvest'는 R 버전 4.1.3에서 작성되었습니다"


[1] "rvest"


Warning message in rbind(c("클라우스", "191613", "애니메이션, 코미디, 가족", "스페인, 영국", :
"number of columns of result is not a multiple of vector length (arg 11)"


In [5]:
final_info[1,]

title 
                                       "클라우스" 
                                             code 
                                         "191613" 
                                            genre 
                       "애니메이션, 코미디, 가족" 
                                          country 
                                   "스페인, 영국" 
                                          runtime 
                                          "96분 " 
                                          release 
                               " 2019.11.15 개봉" 
                                         director 
    "서지오 파블로스, 카를로스 마르티네즈 로페즈" 
                                            actor 
"제이슨 슈왈츠먼, J.K. 시몬스, 라시다 존스더보기" 
                                       view_class 
                             "[국내] 전체 관람가" 
                                  audience_age_10 
                                               NA 
                                  audience_age_20 
                                               NA 
                                  audience_age_30 
                                               NA 
                                  audience_age_40 
                                               NA 
                                  audience_age_50 
                                               NA 
                                    netizen_score 
                                           "9.80" 
                                    netizen_count 
                                            "322" 
                                         ntz_male 
                                           "9.71" 
                                       ntz_female 
                                           "9.88" 
                                           ntz_10 
                                           "9.89" 
                                           ntz_20 
                                           "9.86" 
                                           ntz_30 
                                           "9.79" 
                                           ntz_40 
                                           "9.72" 
                                           ntz_50 
                                           "9.17" 
                                   audience_score 
                                               NA 
                                   audience_count 
                                               NA 
                                    audience_male 
                                               NA 
                                  audience_female 
                                               NA 
                                      audience_10 
                                               NA 
                                      audience_20 
                                               NA 
                                      audience_30 
                                               NA 
                                      audience_40 
                                               NA 
                                      audience_50 
                                               NA